In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
EPOCHS     = 40
BATCH_SIZE = 64

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/content/drive/MyDrive/torch_tutorial/3min_torch/5ch_cnn/.data',
                   train = True,
                   download = True,
                   transform = transforms.Compose([
                                                   transforms.ToTensor(),
                                                   transforms.Normalize((0.1307,),(0.3081))
                   ])),
                   batch_size = BATCH_SIZE, shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('/content/drive/MyDrive/torch_tutorial/3min_torch/5ch_cnn/.data',
                   train = False,
                   transform = transforms.Compose([
                                                   transforms.ToTensor(),
                                                   transforms.Normalize((0.1307,),(0.3081))
                   ])),
                   batch_size = BATCH_SIZE, shuffle=True
)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting /content/drive/MyDrive/torch_tutorial/3min_torch/5ch_cnn/.data/MNIST/raw/train-images-idx3-ubyte.gz to /content/drive/MyDrive/torch_tutorial/3min_torch/5ch_cnn/.data/MNIST/raw




Extracting /content/drive/MyDrive/torch_tutorial/3min_torch/5ch_cnn/.data/MNIST/raw/train-labels-idx1-ubyte.gz to /content/drive/MyDrive/torch_tutorial/3min_torch/5ch_cnn/.data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting /content/drive/MyDrive/torch_tutorial/3min_torch/5ch_cnn/.data/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/drive/MyDrive/torch_tutorial/3min_torch/5ch_cnn/.data/MNIST/raw




Extracting /content/drive/MyDrive/torch_tutorial/3min_torch/5ch_cnn/.data/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/drive/MyDrive/torch_tutorial/3min_torch/5ch_cnn/.data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.conv2_drop = nn.Dropout2d()
    self.fc1 = nn.Linear(320, 50)
    self.fc2 = nn.Linear(50, 10)
    
  def forward(self, x):
    x = F.relu(F.max_pool2d(self.conv1(x), 2))
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2)) # 아직 2차원 
    x = x.view(-1, 320)  # keras의 Flatten (-1 : 남는 모든 차원, 320 : x가 가진 모든 원소의 개수 == vector의 차원 수)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)
    return x

In [6]:
model = ConvNet().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

In [7]:
def train(model, train_loader, optimizer, epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    data, target = data.to(DEVICE), target.to(DEVICE)
    optimizer.zero_grad()
    output = model(data)
    loss = F.cross_entropy(output, target)
    loss.backward()
    optimizer.step()

    if batch_idx % 200 == 0:
      print('Train epoch : {} [{} / {} ({:.0f}%)]\t Loss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()))

In [8]:
def evaluate(model, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(DEVICE), target.to(DEVICE)
      output = model(data)

      test_loss += F.cross_entropy(output, target, reduction='sum').item()

      pred = output.max(1, keepdim=True)[1]
      correct += pred.eq(target.view_as(pred)).sum().item()
  
  test_loss /= len(test_loader.dataset)
  test_accuracy = 100. * correct / len(test_loader.dataset)
  return test_loss, test_accuracy

In [9]:
for epoch in range(1, EPOCHS + 1):
  train(model, train_loader, optimizer, epoch)
  test_loss, test_accuracy = evaluate(model, test_loader)

  print('[{}] TEST LOSS : {:.4f}, ACC : {:.2f}%'.format(epoch, test_loss, test_accuracy))

Train epoch : 1 [0 / 60000 (0%)]	 Loss: 2.388710
Train epoch : 1 [12800 / 60000 (21%)]	 Loss: 1.009802
Train epoch : 1 [25600 / 60000 (43%)]	 Loss: 0.654728
Train epoch : 1 [38400 / 60000 (64%)]	 Loss: 0.509315
Train epoch : 1 [51200 / 60000 (85%)]	 Loss: 0.546422
[1] TEST LOSS : 0.1802, ACC : 94.73%
Train epoch : 2 [0 / 60000 (0%)]	 Loss: 0.454484
Train epoch : 2 [12800 / 60000 (21%)]	 Loss: 0.342151
Train epoch : 2 [25600 / 60000 (43%)]	 Loss: 0.369217
Train epoch : 2 [38400 / 60000 (64%)]	 Loss: 0.384065
Train epoch : 2 [51200 / 60000 (85%)]	 Loss: 0.509669
[2] TEST LOSS : 0.1201, ACC : 96.41%
Train epoch : 3 [0 / 60000 (0%)]	 Loss: 0.258086
Train epoch : 3 [12800 / 60000 (21%)]	 Loss: 0.524943
Train epoch : 3 [25600 / 60000 (43%)]	 Loss: 0.409955
Train epoch : 3 [38400 / 60000 (64%)]	 Loss: 0.394062
Train epoch : 3 [51200 / 60000 (85%)]	 Loss: 0.307407
[3] TEST LOSS : 0.0981, ACC : 96.85%
Train epoch : 4 [0 / 60000 (0%)]	 Loss: 0.227944
Train epoch : 4 [12800 / 60000 (21%)]	 Loss: 